<a href="https://colab.research.google.com/github/hfwalvir/Dissertation/blob/main/Symmetric_key.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cryptography



In [ ]:
from cryptography.fernet import Fernet
import pandas as pd

#Data Extracted from AALI's Balance Sheet
data = {
    'symbol': ['AALI', 'AALI', 'AALI', 'AALI', 'AALI'],
    'account': ['Accounts Payable', 'Accounts Receivable', 'Accumulated Depreciation', 'Additional Paid In Capital', 'Allowance For Doubtful Accounts Receivable'],
    'type': ['BS', 'BS', 'BS', 'BS', 'BS'],
    '2020': [770264000000,  765849000000, 10920950000000, 3878995000000,-24261000000],
    '2021': [1026717000000 , 458135000000, -12133810000000, 3878995000000, -24543000000],
    '2022': [1224423000000 , 848770000000,-13303750000000,3878995000000,-27057000000],
    '2023': [842064000000,674487000000, -14436850000000, 3878995000000, -26516000000]
}

df = pd.DataFrame(data)

# Step 1: AALI generates a symmetric key and encrypts its data
aali_symmetric_key = Fernet.generate_key()
aali_fernet = Fernet(aali_symmetric_key)
encrypted_data = aali_fernet.encrypt(df.to_json().encode())

# Step 2: AALI generates another symmetric key to encrypt the copy of the first symmetric key
key_encryption_key = Fernet.generate_key()
key_encryption_key_copy = bytes(key_encryption_key)  # Create a copy of the key encryption key
key_fernet = Fernet(key_encryption_key)

# Step 3: AALI encrypts a copy of its symmetric key
encrypted_symmetric_key = key_fernet.encrypt(aali_symmetric_key)

# Save the encrypted data and symmetric key to text files
with open('encrypted_data.txt', 'wb') as file:
    file.write(encrypted_data)

with open('encrypted_symmetric_key.txt', 'wb') as file:
    file.write(encrypted_symmetric_key)

with open('key_encryption_key.txt', 'wb') as file:
    file.write(key_encryption_key)

# Step 4: ABDA decrypts the encrypted symmetric key
abda_key_fernet = Fernet(key_encryption_key_copy)
decrypted_symmetric_key = abda_key_fernet.decrypt(encrypted_symmetric_key)

# Step 5: ABDA uses the decrypted symmetric key to decrypt AALI's data
abda_fernet = Fernet(decrypted_symmetric_key)
decrypted_data = abda_fernet.decrypt(encrypted_data)

# Convert decrypted data to json format
decrypted_df = pd.read_json(decrypted_data.decode())

print("Original data:")
print(df)
print("\nDecrypted data:")
print(decrypted_df)

# Verify that the decrypted data matches the original
if df.equals(decrypted_df):
    print("\nSuccessful encryption and decryption: The decrypted data matches the original.")
else:
    print("\nError: The decrypted data does not match the original.")

    # If they don't match, check each column for discrepancies
    for column in df.columns:
        if not df[column].equals(decrypted_df[column]):
            print(f"Discrepancy found in column: {column}")
            print("Original:", df[column])
            print("Decrypted:", decrypted_df[column])
            print()

Original data:
  symbol                                     account type            2020  \
0   AALI                            Accounts Payable   BS    770264000000   
1   AALI                         Accounts Receivable   BS    765849000000   
2   AALI                    Accumulated Depreciation   BS  10920950000000   
3   AALI                  Additional Paid In Capital   BS   3878995000000   
4   AALI  Allowance For Doubtful Accounts Receivable   BS    -24261000000   

             2021            2022            2023  
0   1026717000000   1224423000000    842064000000  
1    458135000000    848770000000    674487000000  
2 -12133810000000 -13303750000000 -14436850000000  
3   3878995000000   3878995000000   3878995000000  
4    -24543000000    -27057000000    -26516000000  

Decrypted data:
  symbol                                     account type            2020  \
0   AALI                            Accounts Payable   BS    770264000000   
1   AALI                         Accoun

<ipython-input-5-dea629fcbb2f>:49: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  decrypted_df = pd.read_json(decrypted_data.decode())
